# Eager Execution

In [1]:
import syft as sy
sy.requires(">=0.8.1-beta")

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/dk/Desktop/projects/PySyft


✅ The installed version of syft==0.8.1b2 matches the requirement >=0.8.1b0


In [2]:
node = sy.orchestra.launch(name="test-domain-1", port=8080, dev_mode=True)

Starting test-domain-1 server on 0.0.0.0:8080

SQLite Store Path:
!open file:///tmp/7bca415d13ed1ec841f0d0aede098dbb.sqlite



INFO:     Started server process [82544]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8080): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [3]:
client = node.login(email="info@openmined.org", password="changethis")
client

<SyftClient - test-domain-1 <7bca415d13ed1ec841f0d0aede098dbb>: HTTPConnection: http://localhost:8080>

In [4]:
guest_client = node.client.guest()
guest_client

<SyftClient - test-domain-1 <7bca415d13ed1ec841f0d0aede098dbb>: HTTPConnection: http://localhost:8080>

In [5]:
assert client.credentials != guest_client.credentials

# Simple permissions

In [6]:
import numpy as np

In [7]:
input_obj = sy.TwinObject(
    private_obj=np.array([[3,3,3], [3,3,3]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)
input_obj

```python
class TwinObject:
  id: str = 327ca2f6f689481dae5843c9ba78d31b
  private_obj: str = [[3 3 3]
 [3 3 3]]
  private_obj_id: str = 108b3036a7b44fd2b9b347e489aa2cc7
  mock_obj: str = [[1 1 1]
 [1 1 1]]
  mock_obj_id: str = 14006f97b3a04b9197514ad589e8a98e

```

In [8]:
# user creates
input_ptr = client.api.services.action.set(input_obj)

# guest gets pointer
pointer = guest_client.api.services.action.get_pointer(input_ptr.id)
pointer.id

<UID: 327ca2f6f689481dae5843c9ba78d31b>

In [9]:
flat_ptr = pointer.flatten()
flat_ptr

```python
TwinPointer(Mock)
```
array([1, 1, 1, 1, 1, 1])

In [10]:
# read permission error
res_guest = guest_client.api.services.action.get(flat_ptr.id)
print(res_guest)
assert not isinstance(res_guest, sy.ActionObject)

Permission: <READ: a59b292940f24050bde8bb676e559339 as 631ebf6b29df6e81c37cd7c5a8a6524c12f521e7cb51e02b58895ed0acfb110e> denied


In [11]:
# get as user with permission
res_root = flat_ptr.get_from(client)
assert all(res_root == [3,3,3,3,3,3])

# Plans

## Plans with object instantiations

In [12]:
@sy.planify
def my_plan(x=np.array([1,2,3,4,5,6])):
    return x+1

In [13]:
plan_ptr = my_plan.send(guest_client)

In [14]:
input_obj = sy.TwinObject(
    private_obj=np.array([1,2,3,4,5,6]),
    mock_obj=np.array([1,1,1,1,1,1])
)

In [16]:
input_obj

```python
class TwinObject:
  id: str = 7eb84a52fcff41daa9d7dad1c0cc7016
  private_obj: str = [1 2 3 4 5 6]
  private_obj_id: str = a1c120461c22430b84c64321722a2bda
  mock_obj: str = [1 1 1 1 1 1]
  mock_obj_id: str = 9db1b2e62e984bd79aa0074191752675

```

In [17]:
_id = client.api.services.action.set(input_obj).id
pointer = guest_client.api.services.action.get_pointer(_id) 

In [20]:
res_ptr = plan_ptr(x=pointer)

In [21]:
res_ptr.get_from(client)

array([2, 3, 4, 5, 6, 7])

## Complex plans

In [22]:
@sy.planify
def my_plan(x=np.array([1,2,3,4,5,6])):
    y = x.flatten() # method -> [1,2,3,4,5,6]
    min_val = x.min() # method -> 1
    s = x.shape # getattribute -> 6    
    w = x[min_val] # __getitem__ -> 2
    y[min_val] = min_val # __setitem__ y = [1,1,3,4,5,6]
    res = client.api.lib.numpy.sum(y) + w + s # client function and __add__ 20 + 2 + 6 = 28
    return res #28

In [23]:
my_plan

```python
Pointer
```
```python
class Plan:
  id: str = 6ef4823826b94ca4b789b4ebbed0c543
  inputs: str = {'x': Pointer:
[1 2 3 4 5 6]}
  outputs: str = [Pointer:
[28]]
  actions: str = [ActionObject NumpyArrayObject[07c..c].flatten(,), ActionObject NumpyArrayObject[07c..c].min(,), ActionObject NumpyArrayObject[07c..c].shape(,), ActionObject NumpyArrayObject[07c..c].__getitem__(056..8,), ActionObject NumpyArrayObject[504..2].__setitem__(056..8, 056..8,), ActionObject numpy.sum(504..2,), ActionObject NumpyScalarObject[5a9..9].__add__(21d..0,), ActionObject NumpyScalarObject[3bc..0].__add__(541..5,)]
  code: str = "@sy.planify
def my_plan(x=np.array([1,2,3,4,5,6])):
    y = x.flatten() # method -> [1,2,3,4,5,6]
    min_val = x.min() # method -> 1
    s = x.shape # getattribute -> 6    
    w = x[min_val] # __getitem__ -> 2
    y[min_val] = min_val # __setitem__ y = [1,1,3,4,5,6]
    res = client.api.lib.numpy.sum(y) + w + s # client function and __add__ 20 + 2 + 6 = 28
    return res #28
"
  client: str = None
  syft_passthrough_attrs: str = ['inputs', 'outputs', 'code', 'actions', 'client']

```

In [24]:
plan_ptr = my_plan.send(guest_client)

In [25]:
plan_ptr

```python
Pointer
```
```python
class Plan:
  id: str = 6ef4823826b94ca4b789b4ebbed0c543
  inputs: str = {'x': Pointer:
[1 2 3 4 5 6]}
  outputs: str = [Pointer:
[28]]
  actions: str = [ActionObject NumpyArrayObject[07c..c].flatten(,), ActionObject NumpyArrayObject[07c..c].min(,), ActionObject NumpyArrayObject[07c..c].shape(,), ActionObject NumpyArrayObject[07c..c].__getitem__(056..8,), ActionObject NumpyArrayObject[504..2].__setitem__(056..8, 056..8,), ActionObject numpy.sum(504..2,), ActionObject NumpyScalarObject[5a9..9].__add__(21d..0,), ActionObject NumpyScalarObject[3bc..0].__add__(541..5,)]
  code: str = "@sy.planify
def my_plan(x=np.array([1,2,3,4,5,6])):
    y = x.flatten() # method -> [1,2,3,4,5,6]
    min_val = x.min() # method -> 1
    s = x.shape # getattribute -> 6    
    w = x[min_val] # __getitem__ -> 2
    y[min_val] = min_val # __setitem__ y = [1,1,3,4,5,6]
    res = client.api.lib.numpy.sum(y) + w + s # client function and __add__ 20 + 2 + 6 = 28
    return res #28
"
  client: str = None
  syft_passthrough_attrs: str = ['inputs', 'outputs', 'code', 'actions', 'client']

```

In [26]:
input_obj = sy.TwinObject(
    private_obj=np.array([1,2,3,4,5,6]),
    mock_obj=np.array([1,1,1, 1,1,1])
)

_id = client.api.services.action.set(input_obj).id
pointer = guest_client.api.services.action.get_pointer(_id) 

In [27]:
res_ptr = plan_ptr(x=pointer)

In [28]:
res_ptr.get_from(client)

array([29])

## Plans with function calls

In [ ]:
client.api.lib.numpy.sum

In [ ]:
@sy.planify
def my_plan(x=np.array([[2,2,2], [2,2,2]])):
    y = x.flatten()
    w = client.api.lib.numpy.sum(y)
    return w

In [ ]:
plan_ptr = my_plan.send(guest_client)

input_obj = sy.TwinObject(
    private_obj=np.array([[3,3,3], [3,3,3]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

input_ptr = client.api.services.action.set(input_obj)
pointer = guest_client.api.services.action.get_pointer(input_ptr.id) 


In [ ]:
res_ptr = plan_ptr(x=pointer)

In [ ]:
assert res_ptr.get_from(client) == 18

## simple execution (root downloads)

In [ ]:
@sy.planify
def my_plan(x=np.array([[2,2,2], [2,2,2]])):
    y = x.flatten()
    z = y.prod()
    return z

In [ ]:
plan_ptr = my_plan.send(guest_client)

In [ ]:
input_obj = sy.TwinObject(
    private_obj=np.array([[3,3,3], [3,3,3]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [ ]:
input_ptr = client.api.services.action.set(input_obj)

In [ ]:
pointer = guest_client.api.services.action.get_pointer(input_ptr.id) 

In [ ]:
res_ptr = plan_ptr(x=pointer)

In [ ]:
# TODO: should be 1
res_ptr

In [ ]:
assert not isinstance(guest_client.api.services.action.get(res_ptr.id), sy.ActionObject)

In [ ]:
assert res_ptr.get_from(client) == np.array([[3,3,3], [3,3,3]]).flatten().prod()

## Downloading the result

In [ ]:
res_ptr.request(guest_client)

In [ ]:
# TODO: fix this issue with custom 

In [ ]:
client.api.services.request[0].approve_with_client(client)

In [ ]:
res_ptr.get_from(guest_client)

# Pointers (Twins)

## setattribute

In [ ]:
private_data, mock_data = np.array(
    [[1.0,2.0,3.0], [4.0,5.0,6.0]]), np.array([[1.0,1.0,1.0], [1.0,1.0,1.0]]
)

In [ ]:
obj = sy.TwinObject(
    private_obj=private_data,
    mock_obj=mock_data
)

In [ ]:
obj_pointer = client.api.services.action.set(obj)
obj_pointer = guest_client.api.services.action.get_pointer(obj_pointer.id)

In [ ]:
original_id = obj_pointer.id

In [ ]:
obj_pointer

In [ ]:
# note that this messes up the data and the shape of the array
obj_pointer.dtype = np.int32  

In [ ]:
res = obj_pointer.get_from(client)

In [ ]:
assert res.dtype == np.int32

In [ ]:
private_data.dtype= np.int32
mock_data.dtype= np.int32

In [ ]:
assert (res == private_data).all()
assert (obj_pointer.syft_action_data == mock_data).all()
assert not (obj_pointer.syft_action_data == private_data).all()

## getattribute

In [ ]:
obj = sy.TwinObject(
    private_obj=np.array([[1,2,3], [4,5,6]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [ ]:
obj_pointer = client.api.services.action.set(obj)
obj_pointer = guest_client.api.services.action.get_pointer(obj_pointer.id)

In [ ]:
size_pointer = obj_pointer.size   

In [ ]:
assert size_pointer.get_from(client) == 6

## methods

In [ ]:
obj = sy.TwinObject(
    private_obj=np.array([[1,2,3], [4,5,6]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [ ]:
obj_pointer = client.api.services.action.set(obj)
obj_pointer = guest_client.api.services.action.get_pointer(obj_pointer.id)

In [ ]:
flat_pointer = obj_pointer.flatten()   

In [ ]:
assert all(flat_pointer.get_from(client) == np.array([1,2,3,4,5,6]))

## Indexing

In [ ]:
obj = sy.TwinObject(
    private_obj=np.array([[1,2,3], [4,5,6]]),
    mock_obj=np.array([[1,1,1], [1,1,1]])
)

In [ ]:
obj_pointer = client.api.services.action.set(obj)
obj_pointer = guest_client.api.services.action.get_pointer(obj_pointer.id)

In [ ]:
indexed_pointer = obj_pointer[0, 0:2]   

In [ ]:
assert all(indexed_pointer.get_from(client) == np.array([1,2]))

In [ ]:
# if node.node_type.value == "python":
node.land()